In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import statsmodels.api as sm
import itertools
import pandas as pd
import collections
from tqdm import tqdm
from joblib import Parallel, delayed
from operator import itemgetter
from itertools import chain

from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression

# Wczytanie danych

In [3]:
# https://www.ue.katowice.pl/fileadmin/_migrated/content_uploads/7_B.Fraczek_Analiza_Czynnikow....pdf
df = pd.read_excel("datasets/cleaned.xlsx", sheet_name = ["Polska"])["Polska"]
df = df.fillna(df.mean())
df.head()

,Rok,Procent dochodu rozporządzalnego przeznaczany na oszczędności,Procent dochodu rozporzązdalnego przeznaczanego na spłatę długów,Zmiana dochodu rozporządzalnego względem roku poprzeniego wyrażona w procentach,Zmiana sumy przeznaczanej na konsumpcję przez gospodarstwa domowe,Jaki procent aktyw gospodarst domowych stanowią gotówka i depozyty,"Rezerwy ubezpieczeń na życie (wyrażone w procetach zobowiązań, których pokrycie w środkach muszą posiadać ubezpieczyciele)",Suma aktyw gospodarstw domowych w dolarach per capita,Przeciętne roczne wynagrodzenie (dane z ZUSu),Średnie stopy procentowe w danym roku,Cena 1 m2 powierzchni użytkowej budynku mieszkalnego oddanego do użytkowania,Średnia stopa bezrobocia,Cena: olej jadalny rafinowany rzepakowy konfekcjonowany - za 1l
0,1999,9.946499,10.721940,3.552158,5.544923,57.25610,2.920997,5176.925700,20480.88,16.444444,2132.50,11.991667,3.30
1,2000,10.153177,11.826970,2.730212,2.897587,60.30187,4.445247,5390.972885,23085.72,20.416667,2281.25,14.008333,2.72
2,2001,11.655554,17.741950,3.903780,2.020035,62.12636,5.267816,5876.964382,24742.20,16.181818,2510.00,16.225000,2.59
3,2002,8.270142,44.478893,0.058289,3.871436,57.49397,5.963031,6643.192654,25598.52,8.593750,2403.50,19.916667,2.36
4,2003,7.302362,19.775300,0.675174,1.381836,42.79832,4.755579,9196.616536,26417.64,6.137500,2238.00,19.508333,2.61


In [4]:
df.set_index("Rok", inplace=True)
df.head()

,Procent dochodu rozporządzalnego przeznaczany na oszczędności,Procent dochodu rozporzązdalnego przeznaczanego na spłatę długów,Zmiana dochodu rozporządzalnego względem roku poprzeniego wyrażona w procentach,Zmiana sumy przeznaczanej na konsumpcję przez gospodarstwa domowe,Jaki procent aktyw gospodarst domowych stanowią gotówka i depozyty,"Rezerwy ubezpieczeń na życie (wyrażone w procetach zobowiązań, których pokrycie w środkach muszą posiadać ubezpieczyciele)",Suma aktyw gospodarstw domowych w dolarach per capita,Przeciętne roczne wynagrodzenie (dane z ZUSu),Średnie stopy procentowe w danym roku,Cena 1 m2 powierzchni użytkowej budynku mieszkalnego oddanego do użytkowania,Średnia stopa bezrobocia,Cena: olej jadalny rafinowany rzepakowy konfekcjonowany - za 1l
Rok,,,,,,,,,,,,
1999,9.946499,10.721940,3.552158,5.544923,57.25610,2.920997,5176.925700,20480.88,16.444444,2132.50,11.991667,3.30
2000,10.153177,11.826970,2.730212,2.897587,60.30187,4.445247,5390.972885,23085.72,20.416667,2281.25,14.008333,2.72
2001,11.655554,17.741950,3.903780,2.020035,62.12636,5.267816,5876.964382,24742.20,16.181818,2510.00,16.225000,2.59
2002,8.270142,44.478893,0.058289,3.871436,57.49397,5.963031,6643.192654,25598.52,8.593750,2403.50,19.916667,2.36
2003,7.302362,19.775300,0.675174,1.381836,42.79832,4.755579,9196.616536,26417.64,6.137500,2238.00,19.508333,2.61


# Opis zmiennych
Oto lista zebranych zmiennych:
- **Procent dochodu rozporządzalnego przeznaczany na oszczędności** – to
zmienna, która w moim modelu opisuje to ile oszczędzają Polacy. Pochodzi ona ze strony https://data.oecd.org/hha/household-savings.htm
-  **Procent dochodu rozporzązdalnego przeznaczanego na spłatę długów** - została wybrana ponieważ stanowi procent dochodu rozporządzalnego.
Zwiększenie/zmniejszenie jej o punkt procentowy spowoduje zwiększenie/zmniejszenie łącznie o punkt procentowy pozostałych zmiennych opisujących procent dochodu rozporządalnego (w tym procentu dochodu prze-
znaczanego na oszczędności).
- **Zmiana dochodu rozporządzalnego względem roku poprzeniego wyrażona w procentach** – wyraźna zmiana dochodu rozporządzalnego może
być niepokojącym sygnałem dla konsumentów, co może spowodować, że
przeznaczą większy/mniejszy procent swojego dochodu rozporządalnego
na oszczędności
- **Zmiana sumy przeznaczanej na konsumpcję przez gospodarstwa** domowe zmianna ta opisuje odpowiedź konsumentów na sytuację na rynku.
- **Jaki procent aktyw gospodarstw domowych** stanowią gotówka i depo-
zyty zmiana tej wartości sygnalizuje, iż gospodarstwa domowe lokują swoje
oszczędności w innych źródłach, może bardziej pewnych lub bardziej ryzykowynych. A może też sygnalizować, iż gospodarstwa domowe spieniężają
swoje oszczędności.
- **Rezerwy ubezpieczeń na życie (wyrażone w procetach zobowiązań, których pokrycie w środkach muszą posiadać ubezpieczyciele)** zmienna ta
nie opisuje gospodarstw domowych, ale ogólną sytuację na rynku/świecie.
W przypadku niepokojów społecznych itp. zmienna ta się zmieni, a więc
reaguje w podobnych sytuacjach oszczędności gospodarstw.
- **Suma aktyw gospodarstw domowych w dolarach per capita** jak powyżej.
- **Przeciętne roczne wynagrodzenie (dane z ZUSu)** – dodana ponieważ
pewna jej część to dochód rozporządzalny.
- **Średnie stopy procentowe w danym roku** – zmienna ta ma wpływ na to ile
pieniędzy pozostaje w kieszeniach sporej części konsumentów posiadającej
kredyty hipoteczne.
- **Cena 1 m2 powierzchni użytkowej budynku** mieszkalnego oddanego do
użytkowania
- **Średnia stopa bezrobocia**
- **Cena: olej jadalny rafinowany rzepakowy konfekcjonowany - za 1l** – olej
wykorzystywany jest w wielu daniach, wzrost jest ceny może zmniejszać
dochód rozporządzalny.

# Dobór zmiennych

In [6]:
# Kolumny z odpowiednią wartością wariancji 
cols = df.columns[df.std() / df.mean() > 0.2].values
cols: list = cols.tolist()
y_col = "Procent dochodu rozporządzalnego przeznaczany na oszczędności" 
cols.remove("Procent dochodu rozporządzalnego przeznaczany na oszczędności")
X_cols = cols

In [8]:
cols, len(cols)

(['Procent dochodu rozporzązdalnego przeznaczanego na spłatę długów',
  'Zmiana dochodu rozporządzalnego względem roku poprzeniego wyrażona w procentach',
  'Zmiana sumy przeznaczanej na konsumpcję przez gospodarstwa domowe',
  'Rezerwy ubezpieczeń na życie (wyrażone w procetach zobowiązań, których pokrycie w środkach muszą posiadać ubezpieczyciele)',
  'Suma aktyw gospodarstw domowych w dolarach per capita',
  'Przeciętne roczne wynagrodzenie (dane z ZUSu)',
  'Średnie stopy procentowe w danym roku',
  'Cena 1 m2 powierzchni użytkowej budynku mieszkalnego oddanego do użytkowania',
  'Średnia stopa bezrobocia',
  'Cena: olej jadalny rafinowany rzepakowy konfekcjonowany - za 1l'],
 10)

## Metoda hellwiga

In [9]:
R = df.corr().loc[X_cols, X_cols] # Macierz korelacji zmiennych
R_0 = df.corr().loc[X_cols, y_col] # Macierz koralacji zmiennyh i zmiennej objaśniającej

H = []
for k, combination in enumerate(list(chain.from_iterable([list(itertools.combinations(X_cols, n)) for n in range(1, len(X_cols) + 1)]))):
    h = []
    for var in combination:
        r = R_0[var]
        denominator = np.sum(np.abs(R.loc[list(combination), var]))
        h.append(r**2/denominator)
    H.append((combination, sum(h)))

best = max(H, key=itemgetter(1))

In [10]:
len(H), len(best[0])

(1023, 2)

In [11]:
best

(('Średnie stopy procentowe w danym roku',
  'Cena: olej jadalny rafinowany rzepakowy konfekcjonowany - za 1l'),
 0.8643742743209077)

## Metoda SelectKBest

In [12]:
X = df.loc[:, cols]
y = df.loc[:, "Procent dochodu rozporządzalnego przeznaczany na oszczędności"]

In [14]:
model = SelectKBest(score_func=f_regression, k=4)
results = model.fit(X, y)
results_df=pd.DataFrame(results.scores_)

names=pd.DataFrame(X.columns)
scored=pd.concat([names,results_df], axis=1)
scored.columns = ["Feature", "Score"]
scored.sort_values(by=['Score']).head(13)

,Feature,Score
3,Rezerwy ubezpieczeń na życie (wyrażone w proce...,0.054981
2,Zmiana sumy przeznaczanej na konsumpcję przez ...,0.068153
1,Zmiana dochodu rozporządzalnego względem roku ...,0.161679
8,Średnia stopa bezrobocia,8.914680
7,Cena 1 m2 powierzchni użytkowej budynku mieszk...,20.977827
4,Suma aktyw gospodarstw domowych w dolarach per...,24.593052
5,Przeciętne roczne wynagrodzenie (dane z ZUSu),25.318753
0,Procent dochodu rozporzązdalnego przeznaczaneg...,30.728382
9,Cena: olej jadalny rafinowany rzepakowy konfek...,36.602687
6,Średnie stopy procentowe w danym roku,69.591481


# Model

In [22]:
X = df.loc[:, best[0]]
X["const"] = 1
y = df.loc[:, "Procent dochodu rozporządzalnego przeznaczany na oszczędności"]

In [23]:
model = sm.OLS(y, X)

In [24]:
results = model.fit()
print(results.summary2())

                                        Results: Ordinary least squares
Model:               OLS                                                           Adj. R-squared:     0.857   
Dependent Variable:  Procent dochodu rozporządzalnego przeznaczany na oszczędności AIC:                78.6878 
Date:                2022-06-02 11:41                                              BIC:                81.8213 
No. Observations:    21                                                            Log-Likelihood:     -36.344 
Df Model:            2                                                             F-statistic:        60.88   
Df Residuals:        18                                                            Prob (F-statistic): 9.75e-09
R-squared:           0.871                                                         Scale:              2.1762  
---------------------------------------------------------------------------------------------------------------
                                